# Maze Traversal
- - -

This notebook generates a maze and then populates it with three autonomous agents.
Each agent leverages a unique strategy for trying to escape the maze.

## The Agents
### The Clueless Walker
The clueless walker simply walks in a straight line until encountering a wall.
Once a wall is hit, the agent tries to turn right. If it can't, then it tries 
to turn left. If it cannot, then it turns around.

### The Wall Follower
The wall follower leverages the [wall following](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Wall_follower) algorithm.

### The Path Finder
The path finder uses the [A* algorithm](https://en.wikipedia.org/wiki/A*_search_algorithm) 
to chart a path to the exit. It does not consider the entrance. 

**Resources**
- [Smoothstep](https://smoothstep.io/)
- Breadth First Search
- Dijkstra's algorithm/Fast Marching Method for solving the Eikonal equation?
- BFS is generalized as Disjkstra, which is generalized as Fast Marching, then as Ordered Upwind method, then as Anisotropic Fast Marching
- Bellman-Ford Algorithm
- Fast Marching Algorithm (FMM), Eikonal equation
- [Maze Art](https://troika.uk.com/work/troika-labyrinth/)
- [Lee Algorithm](https://en.wikipedia.org/wiki/Lee_algorithm)
- [Procedural Content Generation: Mazes](http://pcg.wikidot.com/pcg-algorithm:maze)
- [Wikipedia Maze Generation Algorithms](https://en.wikipedia.org/wiki/Maze_generation_algorithm)
- [Smart Move: Intelligent Path Finding](https://www.gamedeveloper.com/programming/smart-move-intelligent-path-finding)
- [Toward more Realistic Path Finding](https://www.gamedeveloper.com/programming/toward-more-realistic-pathfinding)
- [AI Wisdom A* Articles](http://www.aiwisdom.com/ai_astar.html)
- [Maze Solving Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm)
- [Maze Routing Algorithm](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Maze-routing_algorithm)
- [Shortest Path Algorithms](https://en.wikipedia.org/wiki/Maze-solving_algorithm#Shortest_path_algorithm)

Game Programming Gems 1 (PDF)
- Simple Implementation: Page 248
- Optimized Implementation: Page 279 
- Fuzzy Logic for Video Games: Page 313
- A Neural Net Primer: Page 324

In [291]:
# Load python code.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [292]:
# All imports
from __future__ import annotations

from IPython.display import display
from ipycanvas import Canvas,  hold_canvas, MultiCanvas
import time
from typing import List

from generation.structures import Corner, Point
from generation.maze import Maze
from generation.generators.random_backtracer import generate_maze_walls
from generation.npc import Agent
from generation.renderers.units import AGENT_SIZE, ROOM_SIZE_WIDTH, ROOM_SIZE_HEIGHT
from generation.renderers.wall_drawer import draw_maze
from generation.renderers.agents_renderer import draw_agents
from generation.direction import Direction
from generation.walkers.clueless import clueless_walk
from generation.walkers.wall_follower import wall_follower_walk
from generation.walkers.a_star import find_path as find_path_with_a_star, build_path_walker

In [293]:
# Rendering Functions
def draw_path(path: List[Point], canvas: Canvas, line_color: str) -> None:
  """Renders a list of points as a solid line"""
  canvas_points = []

  # Create an array of tuples for canvas to render in a single draw call.
  print(f'Path has {len(path)} steps')
  for location in path:
    # Find the upper left corner for the room.
    upper_left_corner = Corner(location.x * ROOM_SIZE_WIDTH, location.y * ROOM_SIZE_HEIGHT)

    # Find the midpoint of the room.
    horizontal_offset = ROOM_SIZE_WIDTH/2.0
    vertical_offset = ROOM_SIZE_HEIGHT/2.0
    midpoint = Point(upper_left_corner.x + horizontal_offset, upper_left_corner.y + vertical_offset)

    canvas_points.append((midpoint.x, midpoint.y))
  
  canvas.stroke_style = line_color
  canvas.stroke_lines(canvas_points)

def draw_legend(canvas: Canvas, frame) -> None:
  """Renders a legend for the maze."""
  LINE_HEIGHT = 14
  FIRST_LINE = 20
  HORIZONTAL_OFFSET = 450
  with hold_canvas(canvas):
    canvas.text_baseline = "top"
    canvas.clear()

    # Draw the frame count
    canvas.fill_style = 'black'
    canvas.fill_text(f'Frame: {frame}', HORIZONTAL_OFFSET, FIRST_LINE)

    # Draw Wall Walker Legend
    canvas.fill_style = 'blue'
    canvas.fill_rect(HORIZONTAL_OFFSET,FIRST_LINE+LINE_HEIGHT, AGENT_SIZE)
    canvas.fill_style = 'black'
    canvas.fill_text(f'- Wall Follower', HORIZONTAL_OFFSET + AGENT_SIZE + 5, FIRST_LINE+LINE_HEIGHT)

    # Draw Clueless Walker Legend
    canvas.fill_style = 'green'
    canvas.fill_rect(HORIZONTAL_OFFSET,FIRST_LINE+LINE_HEIGHT*2, AGENT_SIZE)
    canvas.fill_style = 'black'
    canvas.fill_text(f'- Clueless Walker', HORIZONTAL_OFFSET + AGENT_SIZE + 5, FIRST_LINE+LINE_HEIGHT*2)

    # Draw A* Walker Legend
    canvas.fill_style = 'yellow'
    canvas.fill_rect(HORIZONTAL_OFFSET,FIRST_LINE+LINE_HEIGHT*3, AGENT_SIZE)
    canvas.fill_style = 'black'
    canvas.fill_text(f'- Path Finder', HORIZONTAL_OFFSET + AGENT_SIZE + 5, FIRST_LINE+LINE_HEIGHT*3)

In [294]:
# The Main Cell

# 1000/125 = 8 FPS
SLEEP_TIME_SEC:float = 0.125

# Generate a maze.
maze: Maze = Maze(20, 20)
generate_maze_walls(maze)

# Create 4 layers of canvases. 0: Maze, 1: A* Path, 2: Agents, 3: HUD
mc = MultiCanvas(n_canvases=4, width=800, height=400)
display(mc)

# Create NPCs with different strategies
common_starting_point = Point(int(maze.width/2), int(maze.height/2))
wall_follower = Agent('blue')
wall_follower.maze_strategy(wall_follower_walk)
wall_follower.move_to(common_starting_point)
wall_follower.face(Direction.SOUTH)

random_walker = Agent('green')
random_walker.maze_strategy(clueless_walk)
random_walker.move_to(common_starting_point)
random_walker.face(Direction.SOUTH)

# Calculate a path using A*
path_finder = Agent('yellow')
path_finder.move_to(common_starting_point)
path_finder.face(Direction.SOUTH)

found_path, escape_path = find_path_with_a_star(path_finder, maze, maze.exit_cell.location)
if not found_path:
  raise Exception('Failed to find a path.') 

path_walker = build_path_walker(escape_path)
path_finder.maze_strategy(path_walker)

agents = [wall_follower, random_walker, path_finder]

# Initial Render
draw_maze(maze, mc[0])
draw_path(escape_path, mc[1], 'red')
draw_agents(agents, mc[2])

for frame in range(200):
  for agent in agents:
    agent.explore(maze)
  draw_agents(agents, mc[2])
  draw_legend(mc[3],frame)
  time.sleep(SLEEP_TIME_SEC)

Exit cell: Point(x=9, y=19)


MultiCanvas(height=400, width=800)

Path has 87 steps


In [ ]:
# The Main Entry Point
"""
SAVE_ANIMATED_GIF = False

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
thread = RenderingThread(renderer, SAVE_ANIMATED_GIF)

display(canvas)
thread.start()
"""
pass